In [271]:
import pandas as pd
import glob
import datetime
import import_ipynb
from NER import get_entities

In [239]:
data = pd.DataFrame()
# read csv files int to a dataframe 
for file_name in glob.glob('../Data_Collecting/data_files/all_data_streams/'+'*.csv'):
    df = pd.read_csv(file_name,lineterminator='\n')
    data = pd.concat([data, df])

In [240]:
data.shape

(643771, 10)

In [241]:
data.sample(1)

,Unnamed: 0,comment_id,created,fullname,parent_id,subreddit,text,post_id,title,Unnamed: 0.1
291,291,gz2fyp0,2021-05-22 16:13:15,t1_gz2fyp0,t3_niicjm,pennystocks,Technical analysis and astrology are quite si...,NaN,NaN,NaN


In [242]:
data = data.drop(['Unnamed: 0','Unnamed: 0.1', 'title'],axis=1)
data['fullname'] = data['fullname'].apply(lambda x: x[:2])
data.rename(columns = {'fullname':'type'}, inplace = True)
data.rename(columns = {'post_id':'submission_id'}, inplace = True)
data.head(1)

,comment_id,created,type,parent_id,subreddit,text,submission_id
0,gylrunc,2021-05-18 19:10:20,t1,t3_nf66zs,stocks,And it’s all going green again,NaN


In [243]:
data.describe()

,comment_id,created,type,parent_id,subreddit,text,submission_id
count,593201,643771,643771,593201,643771,615296,50570
unique,591032,409071,2,243814,7,539955,5795
top,gzfjyp2,2021-05-17 20:41:50,t1,t3_nmue5k,wallstreetbets,I am a bot from /r/wallstreetbets. Your submis...,nera1s
freq,4,83,593201,18336,555055,3982,83


In [244]:
data.sample(10)

,comment_id,created,type,parent_id,subreddit,text,submission_id
623,gz3qup7,2021-05-22 22:48:39,t1,t3_nintnh,pennystocks,Wow this is definitely my goal,NaN
9811,gz10bvt,2021-05-22 05:25:19,t1,t1_gz106fk,wallstreetbets,Amc better moon or I’m screwed on my calls,NaN
7184,gzoddod,2021-05-27 20:03:36,t1,t3_nmg57b,wallstreetbets,Did not time the markets perfectly today. Off ...,NaN
7393,gynj2lm,2021-05-19 03:17:44,t1,t3_nfklff,wallstreetbets,Today was ev tommorow semiconductors then cann...,NaN
8076,gzrsp4v,2021-05-28 15:56:12,t1,t1_gzrsf0c,wallstreetbets,Bullshit,NaN
3562,gywdq2s,2021-05-21 02:25:48,t1,t1_gywdk4l,wallstreetbets,You're average Masterpiece - that's not losing...,NaN
6062,gzoc4ik,2021-05-27 19:54:41,t1,t1_gzoc1n5,wallstreetbets,True words,NaN
1660,gz6qc46,2021-05-23 18:19:09,t1,t3_ni1fw5,wallstreetbets,Gonna laugh at my corn friends that thought i ...,NaN
7155,NaN,2021-05-23 17:48:39,t3,NaN,wallstreetbets,NaN,njcy4m
7988,gyqy5wg,2021-05-19 21:34:16,t1,t1_gyqxzq1,wallstreetbets,2% up thanks to a shitton of call credit sprea...,NaN


In [245]:
data['subreddit'].value_counts()

wallstreetbets    555055
stocks             35377
investing          22308
StockMarket        14014
pennystocks        12012
algotrading         4004
RobinHood           1001
Name: subreddit, dtype: int64

In [254]:
# Split data into comments and submissions
data_submissions = data[data['type'] == 't3']
data_submissions = data_submissions.drop(['comment_id','parent_id'],axis=1)
data_submissions = data_submissions.dropna()
data_comments = data[data['type'] == 't1']
data_comments = data_comments.drop(['submission_id'],axis=1)
data_comments = data_comments.dropna()

In [261]:
## stock mentions
data_submissions['Organizations'] = data_submissions['text'].apply(get_entities)

In [264]:
data_submissions.to_csv('../Data_Collecting/data_files/data_submissions.csv')

In [265]:
data_comments['Organizations'] = data_comments['text'].apply(get_entities)

In [267]:
# Export new data set to a csv file
data_comments.to_csv('../Data_Collecting/data_files/data_comments.csv')

In [274]:
#data_comments.shape
data_submissions.shape

(22098, 6)

In [287]:
data_submissions['subreddit'].value_counts()

wallstreetbets    17550
stocks             1265
investing          1003
pennystocks         845
algotrading         795
StockMarket         368
RobinHood           272
Name: subreddit, dtype: int64

## A look at the mentions

In [294]:
from collections import Counter

In [340]:
BLACKLIST = ['🚀','spy', 'RemindMe', '’ve', 'NYSE', 'CNBC','flair%3AYOLO', 'Daily Advice Thread](https://www.reddit.com','NightMan', 'FDA','YoY', 'Weekly Earnings Discussion Thread](https://www.reddit.com','Weekly](https://www.reddit.com']
ORGS_TICKER = {"Alibaba":"HKG","AMC":"AMC","Amazon":"AMZN","BlackBerry": "BB","Ford":"FORD","Palantir Technologies":"PLTR","FORD":"FORD", "Lordstown Motors":"RIDE","Virgin Galactic":"SPCE","c3.AI":"AI","Tesla":"TSLA", "General Electric":"GE","Apple":"AAPL","GameStop":"GME","Gamestop":"GME", "Nokia":"NOK"}

In [341]:
org_list = []
for orgs in data_submissions['Organizations']:
    for org in orgs:
        if org not in BLACKLIST:
            if org in ORGS_TICKER:
                org = ORGS_TICKER[org]
                org_list.append(org)
            else:
                 org_list.append(org)
for orgs in data_comments['Organizations']:
    for org in orgs:
        if org not in BLACKLIST:
            if org in ORGS_TICKER:
                org = ORGS_TICKER[org]
                org_list.append(org)
            else:
                 org_list.append(org)

            
# Most frequent mentioned orgs
org_freq = Counter(org_list)
org_freq.most_common(20)

[('AMC', 22301),
 ('GME', 17450),
 ('PLTR', 2655),
 ('AAPL', 2523),
 ('SPCE', 2362),
 ('AMZN', 2252),
 ('FORD', 2162),
 ('AMD', 1714),
 ('TSLA', 1362),
 ('SEC', 1289),
 ('ATH', 1021),
 ('NOK', 1021),
 ('OTM', 892),
 ('EOD', 890),
 ('ITM', 847),
 ('AI', 627),
 ('FOMO', 627),
 ('COIN', 601),
 ('VIX', 560),
 ('BB', 540)]

,created,type,subreddit,text,submission_id,Organizations
30,2021-05-18 19:20:14,t3,stocks,I’m interested in investing into stocks but ha...,nfjkov,[]
104,2021-05-18 19:41:30,t3,stocks,"Hi,\n\nThe reopening of the world is hopefully...",nfk4b2,[ABNB]
273,2021-05-18 20:23:07,t3,stocks,The article in NYT outlines why ATT has all th...,nfl6c8,"[Disney, ATT, NYT, Time Warner, TPG, T-Mobile,..."
301,2021-05-18 20:29:10,t3,stocks,"**PsychoMarket Recap - Tuesday, May 18, 2021**...",nflbtz,"[Powell’s, WDC, Royal Bank of Canada, the Fede..."
322,2021-05-18 20:34:43,t3,stocks,This is a company I follow loosely. They ran a...,nflgyp,[Plymouth]
...,...,...,...,...,...,...
9007,2021-05-19 13:15:11,t3,wallstreetbets,# Welcome to WSB!\n\n\---\n\nAdditional readin...,ng5j36,"[WSB Culture, NFT]"
9008,2021-05-19 13:26:44,t3,wallstreetbets,&#x200B;\n\nhttps://preview.redd.it/6ydt2a0mw2...,ng5tcb,[]
9012,2021-05-19 13:40:03,t3,wallstreetbets,"Just a quick heads up, retards! We are getting...",ng64vr,"[CBS, Fibo, LOT, UWMC](https://imgur.com]"
9019,2021-05-19 14:19:09,t3,wallstreetbets,So the US government is supposed to release go...,ng73bn,"[UAP, Lockheed, Boeing]"


In [149]:
sample = []
for text in data['text'].sample():
    sample = sample + text.split()
print('*')
print(len(sample))

sam = [data['text'].sample(500)]

sample = []
for s in sam:
    sample = sample + str().split()
freq_sample = []
result = {}
df = pd.DataFrame()
print(len(sample))
# loop till string values present in list str
for s in sam:             
  
    # checking for the duplicacy
    if s not in freq_sample:
        freq_sample.append(s) 

     
for i in range(0, len(freq_sample)):

    # count the frequency of each word(present 
    # in str2) in str and print
    df = df.append({
            'sample': freq_sample[i],
            'count': sample.count(freq_sample[i])
    }, ignore_index=True)
    #result[sample.count(freq_sample[i])] = freq_sample[i]


    #print('Frequency of', freq_sample[i], 'is :', sample.count(freq_sample[i]))    

df.sort_values(by=['count'], ascending=False, axis=0)

*
22
0


,count,sample
0,0.0,"5689 BCRX is not a meme stock. But, btw, t..."


In [ ]:


print('* '+file_name)
    data = pd.read_csv(file_name,lineterminator='\n')
    # Transform data
    print('   * Transforming data')
    dfs = transform_reddit_data(data)
    df_submissions = dfs[0]
    df_comments = dfs[1]
    # Persist submissions and comments 
    print('   * Persisting submissions')
    insert_submissions(df_submissions)
    #submissions.append(df_submissions)
    print('   * Persisting comments')
    insert_comments(df_comments)
    #comments.append(df_comments) 

In [ ]:
df_row = pd.concat([df1, df2])

df_row